In [1]:
import numpy as np
import time
import datetime
import numba
import concurrent.futures
from math import lgamma, floor
from numpy.random import Generator, Philox, SeedSequence
from ill_confirm_bias_FNs_03_COARSE_03cnt import confirm_bias_full
np.set_printoptions(suppress=True)



In [2]:
# mp.set_start_method("fork")
start = time.perf_counter()

runs = 2000
numarms = 2
maxalphabeta = 4
# numagents = 50
# minagents = 4
# maxagents = 12
pulls = 1000
payprobs = np.asarray([.499, .5])
tollerance = 0
nsteps = 10
timesteps = 3*10**4

mem_depth = [5, 10, 100, 1000]
mem_snap = [10, 100, 1000, 10000] #these values must be multiples of nsteps to get recorded


networktype = 'errandom'
erprobA = 0.5
erprobB = 2-2*(1-erprobA)**.5
print(f'erprobB = {erprobB}')

mem = 50 # must be greater than max numagents. FULL STOP? if memshare = 0
memshare = 30
memstore = 10

#number of threads
thrds = 10
# numba.set_num_threads(thrds)

for numagents in [4, 6, 9, 12, 15, 25]:
    sg = SeedSequence()
    rgs = [Generator(Philox(s)) for s in sg.spawn(runs)]
    final_mp = np.zeros((runs, 2))
    depth_mp = np.zeros((runs, len(mem_snap)+1, len(mem_depth)))
    print('*********************************************')
    print('*********************************************')
    
    inputs = [erprobB, mem, memshare, memstore, networktype, maxalphabeta, pulls, numagents, numarms, tollerance, payprobs, nsteps, timesteps, rgs]
        
    with concurrent.futures.ProcessPoolExecutor(max_workers=thrds) as executor:
        future_to_confirm_bias_full = {executor.submit(confirm_bias_full, mem_snap, mem_depth, erprobB, mem, memshare, memstore, networktype, maxalphabeta, pulls, numagents, numarms, tollerance, payprobs, nsteps, timesteps, rgs, r): inputs for r in range(runs)}
        for future in concurrent.futures.as_completed(future_to_confirm_bias_full):
            inputs = future_to_confirm_bias_full[future]
            try:
                data = future.result()
            except Exception as exc:
                print(f'generated an exception: y? and {exc}')
            else:
                final_mp[data[3]] = data[0:2]
                depth_mp[data[3]] = data[2]
                if data[3]%500 == 0:
                    print(data)
                    print(f'single run result stored as {final_mp[data[3]]} and depths:')
                    print(depth_mp[data[3]])
    
    
    final_mp = np.asarray(final_mp)
    suc_mp = np.asarray(final_mp[:,0])
    con_mp = np.asarray(final_mp[:,1])
    
    
    np.save(f'NGconsensus_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.npy', con_mp)
    np.save(f'NGsuccess_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.npy', suc_mp)
    np.save(f'NGdepths_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.npy', depth_mp)
    np.savetxt(f'NGconsensus_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.txt', con_mp)
    np.savetxt(f'NGsuccess_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.txt', suc_mp)
#     np.savetxt(f'NGdepths_{networktype}_ifer{erprobA}_size{numagents}_t{tollerance}_mem_s_d_{mem}_{memshare}_{memstore}.txt', depth_mp)

    sucrate = np.sum(suc_mp)/runs
    conrate = np.sum(con_mp)/runs
    
    depth_mean = np.mean(depth_mp, axis=0)
    depth_std = np.std(depth_mp, axis=0)

    print('time:')
    print(datetime.datetime.now())

    print(f'for # agents = {numagents} success rate = {sucrate} consensus rate = {conrate}')
    print('network average memory depths:')
    print(depth_mean)
    print('depths standard deviation')
    print(depth_std)
    print(' ')
    print(' ')
    
finish = time.perf_counter()
print(f'Finished in {round(finish-start,0)/60} minutes')

erprobB = 0.5857864376269049
*********************************************
*********************************************
[1, 1, array([[185.845     , 181.42433333, 116.96133333,   1.739     ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [187.        , 187.        ,   0.        ,   0.        ],
       [194.        , 182.        , 108.        ,   0.        ],
       [178.        , 169.        , 127.        ,   0.        ]]), 0]
single run result stored as [1. 1.] and depths:
[[185.845      181.42433333 116.96133333   1.739     ]
 [  0.           0.           0.           0.        ]
 [187.         187.           0.           0.        ]
 [194.         182.         108.           0.        ]
 [178.         169.         127.           0.        ]]
[1, 1, array([[183.649     , 178.42133333, 106.155     ,   1.163     ],
       [ 11.        ,   0.        ,   0.        ,   0.        ],
       [179.        , 179.        ,   0.        ,   0.        ],
       [188.    

[0, 1, array([[429.643     , 423.24966667, 322.59866667,  23.17666667],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [418.        , 403.        ,   0.        ,   0.        ],
       [436.        , 435.        , 308.        ,   0.        ],
       [422.        , 414.        , 316.        ,   2.        ]]), 1000]
single run result stored as [0. 1.] and depths:
[[429.643      423.24966667 322.59866667  23.17666667]
 [  0.           0.           0.           0.        ]
 [418.         403.           0.           0.        ]
 [436.         435.         308.           0.        ]
 [422.         414.         316.           2.        ]]
[0, 1, array([[426.88      , 419.77133333, 308.496     ,  10.02466667],
       [  6.        ,   0.        ,   0.        ,   0.        ],
       [408.        , 403.        ,   0.        ,   0.        ],
       [410.        , 403.        , 274.        ,   0.        ],
       [432.        , 431.        , 290.        ,  31.        ]]), 15

[1, 1, array([[1192.83166667, 1174.47833333,  892.78966667,   63.173     ],
       [  35.        ,    0.        ,    0.        ,    0.        ],
       [1119.        , 1119.        ,    0.        ,    0.        ],
       [1166.        , 1166.        ,  927.        ,    0.        ],
       [1131.        , 1131.        ,  828.        ,   21.        ]]), 0]
single run result stored as [1. 1.] and depths:
[[1192.83166667 1174.47833333  892.78966667   63.173     ]
 [  35.            0.            0.            0.        ]
 [1119.         1119.            0.            0.        ]
 [1166.         1166.          927.            0.        ]
 [1131.         1131.          828.           21.        ]]
[0, 1, array([[1183.93066667, 1162.83333333,  844.09433333,   35.13566667],
       [  95.        ,    0.        ,    0.        ,    0.        ],
       [1146.        , 1112.        ,    0.        ,    0.        ],
       [1177.        , 1148.        ,  539.        ,    0.        ],
       [1215.   